# Homework Week 8
## Neural Networks and Deep Learning : Dino or Dragon?

### Downloading the image dataset of dinos and dragons.

In [1]:
# !wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
# !unzip dino-dragon.zip

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale=1./255)

train_ds = train_generator.flow_from_directory(
    './train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 1594 images belonging to 2 classes.


In [3]:
train_ds.class_indices

{'dino': 0, 'dragon': 1}

In [5]:
test_generator = ImageDataGenerator(rescale=1./255)

test_ds = test_generator.flow_from_directory(
    './test',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 394 images belonging to 2 classes.


### Creating a base model for the CNN

In [6]:
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))


# model.compile(optimizer=keras.optimizers.SGD(lr=0.002, momentum=0.8),
#               loss=keras.losses.BinaryCrossentropy(),
#               metrics=['accuracy'])
# model.summary()
# history = model.fit(
#     train_ds,
#     epochs=10,
#     validation_data=test_ds
# )

### Question 1
Since we have a binary classification problem, what is the best loss function for us?

``binary crossentropy``

### Question 2
What's the total number of parameters of the model?

In [8]:
model = models.Sequential()
inputs = keras.Input(shape=(150, 150, 3))
base = model(inputs)
vectors = keras.layers.Conv2D(32, (3, 3), activation='relu')(base)
pooling = keras.layers.MaxPooling2D((2, 2))(vectors)
flatten = keras.layers.Flatten()(pooling)
inner = keras.layers.Dense(64, activation='relu')(flatten)
outputs = keras.layers.Dense(1, activation='sigmoid')(inner)

optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)
loss = keras.losses.BinaryCrossentropy()
    
model = keras.Model(inputs, outputs)

model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

model.summary()

Model: "model"
_________________________________________________________________


2022-11-22 00:45:21.637485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/nineve/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 sequential (Sequential)     multiple                  0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
          

### Question 3
What is the median of training accuracy for all the epochs for this model?

In [9]:
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
80/80 [==============================] - 23s 275ms/step - loss: 0.6185 - accuracy: 0.6292 - val_loss: 0.5314 - val_accuracy: 0.7386
Epoch 2/10
80/80 [==============================] - 23s 292ms/step - loss: 0.4974 - accuracy: 0.7616 - val_loss: 0.4372 - val_accuracy: 0.8274
Epoch 3/10
80/80 [==============================] - 23s 289ms/step - loss: 0.4138 - accuracy: 0.8143 - val_loss: 0.3858 - val_accuracy: 0.8426
Epoch 4/10
80/80 [==============================] - 20s 250ms/step - loss: 0.3741 - accuracy: 0.8400 - val_loss: 0.4339 - val_accuracy: 0.7919
Epoch 5/10
80/80 [==============================] - 20s 248ms/step - loss: 0.3301 - accuracy: 0.8708 - val_loss: 0.3511 - val_accuracy: 0.8579
Epoch 6/10
80/80 [==============================] - 19s 242ms/step - loss: 0.3074 - accuracy: 0.8802 - val_loss: 0.3291 - val_accuracy: 0.8503
Epoch 7/10
80/80 [==============================] - 19s 238ms/step - loss: 0.2680 - accuracy: 0.8996 - val_loss: 0.3301 - val_accuracy: 0.8503

In [10]:
import numpy as np
np.median(history.history['accuracy'])

0.8754705190658569

### Question 4
What is the standard deviation of training loss for all the epochs for this model?

In [11]:
np.std(history.history['loss'])

0.12779002869173225

## Data Augmentation

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest'
)

train_ds = train_generator.flow_from_directory(
    './train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True,
)

Found 1594 images belonging to 2 classes.


In [13]:
test_ds = train_generator.flow_from_directory(
    './test',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 394 images belonging to 2 classes.


In [14]:
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
80/80 [==============================] - 29s 356ms/step - loss: 0.4635 - accuracy: 0.7823 - val_loss: 0.3842 - val_accuracy: 0.8426
Epoch 2/10
80/80 [==============================] - 28s 355ms/step - loss: 0.4304 - accuracy: 0.8099 - val_loss: 0.3934 - val_accuracy: 0.8325
Epoch 3/10
80/80 [==============================] - 28s 352ms/step - loss: 0.4216 - accuracy: 0.8080 - val_loss: 0.3750 - val_accuracy: 0.8452
Epoch 4/10
80/80 [==============================] - 29s 360ms/step - loss: 0.4069 - accuracy: 0.8199 - val_loss: 0.3527 - val_accuracy: 0.8477
Epoch 5/10
80/80 [==============================] - 28s 344ms/step - loss: 0.3793 - accuracy: 0.8312 - val_loss: 0.4144 - val_accuracy: 0.8122
Epoch 6/10
80/80 [==============================] - 27s 343ms/step - loss: 0.3905 - accuracy: 0.8306 - val_loss: 0.4742 - val_accuracy: 0.7614
Epoch 7/10
80/80 [==============================] - 27s 337ms/step - loss: 0.3831 - accuracy: 0.8243 - val_loss: 0.3927 - val_accuracy: 0.8325

### Question 5
What is the mean of test loss for all the epochs for the model trained with augmentations?

In [15]:
np.mean(history.history['val_loss'])

0.38997604250907897

### Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [16]:
np.mean(history.history['val_accuracy'][5:])

0.8203045725822449